In [59]:
import scanpy as sc
import pandas as pd
import squidpy as sq
import numpy as np
from scipy.spatial import *
from sklearn.preprocessing import *

from sklearn.metrics import *
from scipy.spatial.distance import *

In [60]:
def fx_1NN(i,location_in):
        location_in = np.array(location_in)
        dist_array = distance_matrix(location_in[i,:][None,:],location_in)[0,:]
        dist_array[i] = np.inf
        return np.min(dist_array)
    

def fx_kNN(i,location_in,k,cluster_in):

    location_in = np.array(location_in)
    cluster_in = np.array(cluster_in)


    dist_array = distance_matrix(location_in[i,:][None,:],location_in)[0,:]
    dist_array[i] = np.inf
    ind = np.argsort(dist_array)[:k]
    cluster_use = np.array(cluster_in)
    if np.sum(cluster_use[ind]!=cluster_in[i])>(k/2):
        return 1
    else:
        return 0

In [61]:
def _compute_CHAOS(clusterlabel, location):

        clusterlabel = np.array(clusterlabel)
        location = np.array(location)
        matched_location = StandardScaler().fit_transform(location)

        clusterlabel_unique = np.unique(clusterlabel)
        dist_val = np.zeros(len(clusterlabel_unique))
        count = 0
        for k in clusterlabel_unique:
            location_cluster = matched_location[clusterlabel==k,:]
            if len(location_cluster)<=2:
                continue
            n_location_cluster = len(location_cluster)
            results = [fx_1NN(i,location_cluster) for i in range(n_location_cluster)]
            dist_val[count] = np.sum(results)
            count = count + 1

        return np.sum(dist_val)/len(clusterlabel)
def _compute_PAS(clusterlabel,location):
    
    clusterlabel = np.array(clusterlabel)
    location = np.array(location)
    matched_location = location
    results = [fx_kNN(i,matched_location,k=10,cluster_in=clusterlabel) for i in range(matched_location.shape[0])]
    return np.sum(results)/len(clusterlabel)
    

def markerFC(adata_valid,marker_list,sdm_key):
    rst_dict = {}
    sdm_unique = adata_valid.obs[sdm_key].cat.categories
    for marker in marker_list:
        mean_exp_list = []
        for sdm in sdm_unique:
            mean_exp_list.append(np.mean(adata_valid[adata_valid.obs[sdm_key]==sdm][:,marker].X))
        max_sdm_idx = np.argmax(mean_exp_list)
#         print(sdm_unique[max_sdm_idx])

        max_sdm_value = np.max(mean_exp_list)
        other_sdm_value = np.mean(adata_valid[adata_valid.obs[sdm_key]!=sdm_unique[max_sdm_idx]][:,marker].X)
        cur_fc = max_sdm_value/other_sdm_value
        rst_dict[marker] = cur_fc
    return rst_dict

In [62]:
def compute_ARI(adata,gt_key,pred_key):
    return adjusted_rand_score(adata.obs[gt_key],adata.obs[pred_key])

def compute_NMI(adata,gt_key,pred_key):
    return normalized_mutual_info_score(adata.obs[gt_key],adata.obs[pred_key])

def compute_CHAOS(adata,pred_key,spatial_key='spatial'):
    return _compute_CHAOS(adata.obs[pred_key],adata.obsm[spatial_key])

def compute_PAS(adata,pred_key,spatial_key='spatial'):
    return _compute_PAS(adata.obs[pred_key],adata.obsm[spatial_key])

def compute_ASW(adata,pred_key,spatial_key='spatial'):
    d = squareform(pdist(adata.obsm[spatial_key]))
    return silhouette_score(X=d,labels=adata.obs[pred_key],metric='precomputed')

def compute_HOM(adata,gt_key,pred_key):
    return homogeneity_score(adata.obs[gt_key],adata.obs[pred_key])

def compute_COM(adata,gt_key,pred_key):
    return completeness_score(adata.obs[gt_key],adata.obs[pred_key])

def marker_score(adata,domain_key,top_n=5):
    adata = adata.copy()
    count_dict = adata.obs[domain_key].value_counts()
    adata = adata[adata.obs[domain_key].isin(count_dict.keys()[count_dict>3].values)]
    sc.pp.normalize_per_cell(adata)
    sc.pp.log1p(adata)
    sc.tl.rank_genes_groups(adata,groupby=domain_key)
    selected_genes = []
    for i in range(top_n):
        toadd = list(adata.uns['rank_genes_groups']['names'][i])
        selected_genes.extend(toadd)
    selected_genes = np.unique(selected_genes)
    sq.gr.spatial_neighbors(adata)
    sq.gr.spatial_autocorr(
        adata,
        mode="moran",
        genes=selected_genes,
        n_perms=100,
        n_jobs=1,
    )
    sq.gr.spatial_autocorr(
        adata,
        mode="geary",
        genes=selected_genes,
        n_perms=100,
        n_jobs=1,
    )
    moranI = np.median(adata.uns["moranI"]['I'])
    gearyC = np.median(adata.uns["gearyC"]['C'])
    return moranI,gearyC

In [63]:
adata=sc.read_h5ad('/home/workspace2/zhaofangyuan/domain_output/SpaGCN_without/10/SpaGCN_without_151507.h5ad')
adata_valid = adata[np.logical_not(adata.obs['Region'].isna())]#去除NAN
adata_valid

View of AnnData object with n_obs × n_vars = 4221 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'Region', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5', 'pred_6', 'pred_7', 'pred_8', 'pred_9', 'pred_10', 'pred_refined_1', 'pred_refined_2', 'pred_refined_3', 'pred_refined_4', 'pred_refined_5', 'pred_refined_6', 'pred_refined_7', 'pred_refined_8', 'pred_refined_9', 'pred_refined_10'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'ari', 'memory', 'refined_ari', 'spatial', 'time'
    obsm: 'spatial'

In [64]:
#得到含有pred的csv文件
# adata_valid.obs['pred_1'].to_csv('pred_1.csv',index=None)

In [65]:
result_adata=adata_valid.copy()

In [67]:
#得到新的adata
pred=pd.read_csv('new_method.txt')
# domain=pred['new_method1'].tolist()#txt文件中的第一行是名字
domain=pred.iloc[:,0].tolist()
result_adata.obs['pred']=domain
result_adata.obs['pred']=result_adata.obs['pred'].astype('category')

In [68]:
#computeari
ari=compute_ARI(result_adata,f'Region',f'pred')
ari

0.42130812976368504

In [69]:
#compute nmi
nmi=compute_NMI(result_adata,f'Region',f'pred')
nmi

0.550818750430498

In [70]:
#compute CHAOS
chaos=compute_CHAOS(result_adata,f'pred')
chaos

0.06066783744035732

In [71]:
#compute PAS
pas=compute_PAS(result_adata,f'pred',spatial_key='spatial')
pas

0.17720919213456526

In [72]:
#compute ASW
asw=compute_ASW(result_adata,f'pred',spatial_key='spatial')
asw

-0.029425595396403586

In [73]:
#compute HOM
hom=compute_HOM(result_adata,f'Region',f'pred')
hom

0.5419495519799414

In [74]:
#compute COM
com=compute_COM(result_adata,f'Region',f'pred')
com

0.5599830739241538

In [75]:
#compute marker_score
moranI,gearyC=marker_score(result_adata,f'Region')
print(moranI,gearyC)

/home/workspace2/zhaofangyuan/anaconda3/envs/zhuanhuan/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


  0%|          | 0/100 [00:00<?, ?/s]

  0%|          | 0/100 [00:00<?, ?/s]

0.277929965839376 0.7225470532588847


In [76]:
output_df = pd.DataFrame([[nmi,hom,com,chaos,pas,asw,moranI,gearyC]],
               index = [pred.columns[0]],
               columns=[['Accuracy','Accuracy','Accuracy','Continuity','Continuity','Continuity','Marker score','Marker score'],
                        ['NMI','HOM','COM','CHAOS','PAS','ASW','Moran\'I','Geary\'s C']])
output_df.to_excel('./download_output.xlsx')